In [0]:
"""
Set up Spark access to Azure Data Lake Storage (ADLS) 
using the account access key and list all files in the lakehouse container 
to verify the connection.
"""

spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60106541.dfs.core.windows.net",
    "<AccountKey>"
)

display(
    dbutils.fs.ls("abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/")
)


path,name,size,modificationTime
abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/,gold/,0,1762463595000
abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/processed/,processed/,0,1762430158000
abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/raw/,raw/,0,1762084980000


In [0]:
"""
Install all required Python libraries for text processing and feature engineering. 
Includes NLTK for sentiment analysis, Sentence Transformers for embeddings, 
TextStat for readability metrics, and TextBlob for subjectivity analysis.
"""

%pip install nltk
%pip install sentence-transformers
%pip install textstat
%pip install textblob

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/176.4 kB ? eta -:--:--
   ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/176.4 kB 914.6 kB/s eta 0:00:01
   ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/176.4 kB 914.6 kB/s eta 0:00:01
   ━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/176.4 kB 533.0 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 92.2/176.4 kB 690.5 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 92.2/176.4 kB 690.5 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺ 174.1/176.4 kB 842.0 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 797.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.1 MB ? eta -:--:--
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2

In [0]:
"""
Import all required libraries for Spark, text processing, and feature engineering. 
Includes Spark SQL and ML components for data manipulation and TF-IDF, 
NLTK, TextBlob, TextStat, and Sentence Transformers for NLP tasks, 
and NumPy, Pandas, and scikit-learn for data analysis and vectorization.
"""

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    col, year, month, dayofweek, to_timestamp,
    length, size, split, udf
)

from pyspark.sql.types import (
    StringType, DoubleType, FloatType, ArrayType
)
from pyspark.ml.feature import (
    Tokenizer, StopWordsRemover, CountVectorizer, IDF
)
from pyspark.ml import Pipeline

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import textstat
from sentence_transformers import SentenceTransformer

import numpy as np
import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer


2025-11-19 16:26:15.669303: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-19 16:26:15.672741: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-19 16:26:15.681141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-19 16:26:15.696793: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-19 16:26:15.701488: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-19 16:26:15.715333: I tensorflow/core/platform/cpu_feature_gu

In [0]:
"""
Load the cleaned dataset from the Gold layer (features_v1) stored in Delta format. 
Display the schema to verify column types and show a sample of the first ten records.
"""

df = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/features_v1/"
)

df.printSchema()
display(df.limit(10))


root
 |-- book_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- review_length_words: integer (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- num_reviews: long (nullable = true)



book_id,review_id,title,author_id,name,user_id,rating,review_text,language,n_votes,date_added,review_length_words,avg_rating,num_reviews
25451555,98f1e3d1c5d7aaeea00f61edfc9f21f1,"Saga, Vol. 5 (Saga, #5)",254096,Shelley Lotz,9d99d1c0336449e5513e2b5a4a8703fc,5.0,"ready for vol 6. i have a feeling that this series is going to make me very angry at some point, probably in the near future, but i love it all the same.",eng,31639,2015-09-09,33,4.41334768568353,929
969392,1dc9bf1935df4000e23d8f2ad441bb0a,I Was Amelia Earhart,14245927,Neeraj Maurya,91da79515b78fa33ffb12e531378bc12,4.0,"planes used to be vehicles for dreaming. they were strong and curvaceous, manly and womanly at the same time, simple, almost oldfashioned mechanical toys and vessels carrying the future. as soon as you saw a plane, you started dreaming. it was a thrill just to catch a glimpse of one. a short book whose story is an imaginative recreation of amelia earharts last flight, and its aftermath. the blurb on the back cover mentions its similarity to a j.g. ballard story and it did remind me quite a lot of the unlimited dream company. the tale of a lost pilot, the dreamlike atmosphere of events after the crash, the uncertainty about whether events are really happening or just the fantasy of a dying pilot, are all reminiscent of ballards writings.",,26,null,130,4.0,1
18335634,2b072a6452b7db2af8fbba28d73ed49a,"Clockwork Princess (The Infernal Devices, #3)",2937259,Antinio Ramos Rosa,0e15c13a49a7222fa80b3dbd449408c5,4.0,"on reflection after reading the last book in the mortal instruments series i really realized how much better this series was. i miss tess, will and james and the rest. i honestly dont remember what happened in this book too well, i definitely should have gotten around to writing that review a lot sooner.",eng,167406,2013-09-05,54,4.649129746835443,2528
9455718,fb7423338f069ebef5295d1f118b9cd6,Valdez Is Coming: A Novel,16203678,Gita Yuliani K.,732dcdc2f59d2eb7a524e16d80a42765,3.0,good western story.,en-US,179,2009-10-13,3,3.6666666666666665,3
6194,8286b24f0e96de853b6a7f1af9520d90,Waiting for the Barbarians,8303743,Paul J. Fleming,d5adbbf864f0889921d1ebddec70ed03,3.0,3.5 i was required to read this for class and for some reason its good but weird.,eng,15187,1999-10-01,17,4.0,131
13539044,48f28bcab8086bb4307545085af5eb22,The Silver Linings Playbook,7816721,Estefania Jimenez,11bacecfeb377ca6e14cff643f02024f,5.0,"i did not want to put this down. i read every word, and my only complaint is that it wasnt longer. fantastic book about love, mental illness, and set near my hometown, which is always fun. a great book for just about anyone.",eng,112224,null,43,4.002807411566535,1781
15801668,e1f2e99b29743b0b5a0bbe173e7596c8,The Girls of Atomic City: The Untold Story of the Women Who Helped Win World War II,13468007,Dex O'Donald,ec659dffbcc498d30d2905a084fa0faa,4.0,"using diaries and personal interviews, the author constructs the story of the young women recruited to work at oak ridge, tennessee on a top secret project during world war ii. though they didnt know it at the time, they were working to perfect the atomic bomb, which indeed did bring a close to the war in japan. the story is well told, although i did find the skipping around in time a bit difficult to follow. i enjoyed the fact that the book focused on the unknown young women and highlighted their contribution to the war effort in the united states. weve all heard of rosie the riveter, but not much about these women. applause to the author for telling their story.",eng,16026,2013-03-05,122,3.6132930513595167,331
8874858,a4bba4f59cd044342503027fa5fa71a1,نحو أصول جديدة للفقه الإسلامي: فقه المرأة,4821816,Regina Louro,1ae37b769860aef03cd150bff28998d4,3.0,"mqdm lktb mlyy blm lmntq ldhy qd ybdw gyr mrtbt mbshr blktb, fqd qdm shhrwr qr fy mfhym lsyrwr wlty mthlh bltrykh, wlsyrwr wlty mthlh blttwr, wlkynwn lty mthlh blmjtm lnsny, fhdhh lmfhym tntj mjtmt dynywy thlthy lbd, m fy ljn fhnk bdyn fqt l gyy fyh bdm

In [0]:
"""
Split the dataset into training, validation, and test subsets using a 70/15/15 ratio. 
Count the number of samples in each split and calculate the percentage distribution 
to confirm the split proportions.
"""

train, val, test = df.randomSplit([0.7, 0.15, 0.15], seed=42)

train_count = train.count()
val_count = val.count()
test_count = test.count()

total = train_count + val_count + test_count

train_pct = (train_count / total) * 100
val_pct = (val_count / total) * 100
test_pct = (test_count / total) * 100

print(f"Train: {train_pct:.2f}% | Validation: {val_pct:.2f}% | Test: {test_pct:.2f}%")


Train: 70.01% | Validation: 14.99% | Test: 15.00%


In [0]:
"""
Save the train, validation, and test DataFrames to the Gold layer in Delta format. 
Each split is stored in a separate folder for consistent access and reuse 
during model training and evaluation.
"""

train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/train/"
)

val.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/validation/"
)

test.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/test/"
)


In [0]:
"""
Load the training dataset from the Gold layer (feature_v2/train) stored in Delta format. 
Display the first ten records to verify successful loading and data integrity.
"""

train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/train/"
)

display(train.limit(10))


book_id,review_id,title,author_id,name,user_id,rating,review_text,language,n_votes,date_added,review_length_words,avg_rating,num_reviews
10000761,5dcb02a5a8228a2be4e6fac601907daa,Render Unto Rome: The Secret Life of Money in the Catholic Church,7373144,Roopa Weber,b77f34246ac69b4b66bd67ce93b54677,3.0,"a fascinating read, chalk full of details that benefit from a familiarity with church hierarchy and structure. the subtitle is a tad misleading as this books primary focus is the monetary impact of the clergy sex abuse scandal on the catholic church in america with specific attention to the archdioceses of boston, cleveland and los angeles, but the narrative does extend beyond the u.s. i would recommend it to anyone whos interested in the issues facing the modern church and the need for greater transparency in the institutional structure.",en-GB,136,2011-06-07,89,3.5,4
10000761,91898414e53b123a056e81fc5fcd72c8,Render Unto Rome: The Secret Life of Money in the Catholic Church,7373144,Roopa Weber,6cff11a0530eeddebe0a053337b5513b,3.0,"interesting subject. basically it boils down to the vatican expecting the faithful to just pray, pay and obey. book seemed to ramble in many places. would rec.",en-GB,136,2011-06-07,27,3.5,4
10000761,a13072a0af22520144112fd876302867,Render Unto Rome: The Secret Life of Money in the Catholic Church,7373144,Roopa Weber,a45fb5d39a6a9857ff8362900790510a,3.0,"the author addresses a history of the finances of the vatican and the catholic with a side trip into sex scandals. there is some antidotal information. very little positive information is given about church hierarchy and finance, leaving me the impression that the author is out to tar the leadership and priests of the church with the human flaws of a few. it appears to be well documented and gives some history of the institution. also has misinformation about some church fraternal organizations i.e. the knights of columbus as to their purpose.",en-GB,136,2011-06-07,92,3.5,4
10000761,abae93ad132fef34ffa6c46fec8c5e2a,Render Unto Rome: The Secret Life of Money in the Catholic Church,7373144,Roopa Weber,3ec16d3d84ccd6e92a5669b9f39a9b8f,5.0,"jason berry has delved into the corruption of the r.c. church from the pope down to the local parish priest. he blames the corruption on the unnatural life of celibacy which essentially denies priests an outlet for their sexual inclinations and tends to encourage a gay subculture and opportunities for molestation of the young. this situation resulted in the prosecution of many malefactors among the clergy since 1980 and the loss of millions in civil cases. special attention is given to pope john pauls favorite mexican born priest, maciel, who was able to produce huge donations for the pope and himself but who left behind a horrible heap of corruption. whatever you thought when you picked the book up, you end up knowing its worse than you ever thought possible. it helps to have a background in the catholic religion to know the specifics but anyone can follow the story. its clearly written although very complex.",en-GB,136,2011-06-07,156,3.5,4
1000096,88704179525c63452399ef68ad66754b,"Strontium Dog: Search/Destroy Agency Files, Vol. 1",4749354,Ricardo Cavolo,18ee50445cbff3a198b9bca59ad4a3f9,3.0,"solid esquerra fun whole time through yes, im ignoring those god awful annual bits that were here. but i have to say.... the stories are a bit repetitive. strontium dogs come, are not liked, they look for the criminal, they find, they fight, they collect the reward. but the esquerra art is brilliant.",eng,120,2007-01-15,53,3.0,2
1000096,915852f3eb3a3515a31f9c7d9a9e44ab,"Strontium Dog: Search/Destroy Agency Files, Vol. 1",4749354,Ricardo Cavolo,68b3eb9b8de7e55a6e09c2ee002c1104,3.0,"the first 2000ad reprint book ive bought. i tend to assume ill have read most of them already, by way of titan collections and the various best of 2000ad reprint mags i loved 2000ad extreme edition while it lasted, even though ive never bought the origin

In [0]:
"""
Define and apply a text cleaning function to preprocess review text. 
The function converts text to lowercase, removes URLs, numbers, punctuation, 
and extra spaces. It also filters out very short reviews and adds 
two new columns for word and character counts before saving the cleaned dataset.
"""

@F.udf(returnType=StringType())
def clean_text(text):
    if not text:
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", " url ", text)
    text = re.sub(r"\d+", " num ", text)
    text = re.sub(r"[^a-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

clean_train = train.withColumn("clean_review", clean_text(F.col("review_text")))
clean_train = clean_train.filter(F.length("clean_review") > 10)

clean_train = clean_train.withColumn("review_length_words", size(split(col("clean_review"), " "))) \
                   .withColumn("review_length_chars", length(col("clean_review")))

display(clean_train.limit(10))

clean_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/clean_train/"
)


book_id,review_id,title,author_id,name,user_id,rating,review_text,language,n_votes,date_added,review_length_words,avg_rating,num_reviews,clean_review,review_length_chars
10001021,16b1077b60e3674e4edaef7951194c43,"Dragon Ball, Vol. 1: Son Goku (Dragon Ball, #1)",8841094,hellodickspeight,ebc700f4f7abb6203ef6f8d987d80e40,4.0,"podem ler a opiniao completa aqui. even though the anime was never one of my favourites, i always wanted to read the manga, and so when this got published in portugal, i went ahead and bought it. im glad i did! the drawings are very good, the story is as funny as i remember, and gokus innocence just gets me everytime. ill definetly keep buying.",por,28,2010-10-01,65,3.75,4,podem ler a opiniao completa aqui even though the anime was never one of my favourites i always wanted to read the manga and so when this got published in portugal i went ahead and bought it im glad i did the drawings are very good the story is as funny as i remember and gokus innocence just gets me everytime ill definetly keep buying,336
10001021,3c604adc11027e4621d0d1c189e3f573,"Dragon Ball, Vol. 1: Son Goku (Dragon Ball, #1)",8841094,hellodickspeight,873f794f39e4490562e0e85845e1059e,4.0,"lembrome de ver a serie televisiva quando era mais pequena, na sic, e quando vi a manga a venda mal quis acreditar. d acho que dei pulos de contente, devo ter gritado um valente yay! e a slayra deve ter pensado ou melhor, deve ter tido a certeza que eu havia ensandecido. infelizmente so via os volumes mais recentes, mas veio a feira e o meu irmao talvez ainda mais doido que eu pela serie la fez o favor de comprar. apesar de fa, confesso que pouco me lembro de cada episodio, tenho mais presente a serie dragon ball z, mas pareceme que a manga segue mais ou menos a mesma historia. tem as mesmas piadas, as mesmas personagens, o mesmo desenho... nao sera a 8a maravilha do mundo, mas entretem como os desenhos animados o fizeram em tempos. p eu nao me lembrava era que havia tanta piada com cuecas... revira os olhos vale o dinheiro gasto talvez nao seja para toda a gente, mas va la, quem nao cresceu maravilhado com as aventuras do son goku? p",por,28,2010-10-01,180,3.75,4,lembrome de ver a serie televisiva quando era mais pequena na sic e quando vi a manga a venda mal quis acreditar d acho que dei pulos de contente devo ter gritado um valente yay e a slayra deve ter pensado ou melhor deve ter tido a certeza que eu havia ensandecido infelizmente so via os volumes mais recentes mas veio a feira e o meu irmao talvez ainda mais doido que eu pela serie la fez o favor de comprar apesar de fa confesso que pouco me lembro de cada episodio tenho mais presente a serie dragon ball z mas pareceme que a manga segue mais ou menos a mesma historia tem as mesmas piadas as mesmas personagens o mesmo desenho nao sera a num a maravilha do mundo mas entretem como os desenhos animados o fizeram em tempos p eu nao me lembrava era que havia tanta piada com cuecas revira os olhos vale o dinheiro gasto talvez nao seja para toda a gente mas va la quem nao cresceu maravilhado com as aventuras do son goku p,925
10007061,478f8d299aaa68c4857b1c63cf6f2ea1,Gli amanti dello scandalo/2,4355365,Ephrem the Syrian,4ff550ef0be4d6aeb62c7040b8acd594,3.0,"spoiler alert individuo sempre gli stessi difetti in questautrice. la schone e senza dubbio una scrittrice capace, con un suo stile peculiare, ma per la terza volta sono costretta a rimarcare le stesse mancanze, o forse sarebbe meglio definirle sovrabbondanze. forse in questa seconda parte ce meno sesso, piu amore e piu dramma rispetto alla prima parte, ma si ricade sempre negli stessi difetti sono troppe le scene di sesso gratuito e senza coerenza nel senso del racconto, messe li quasi a forza, dove invece semplici gesti di tenerezza sarebbero bastati per rendere piu vero il momento profondo e delicato che viene narrato tali scene rendono freddi e poco spontanei alcuni passaggi che emotivamente potevano essere molto intensi e profondi, e mi riferisc

In [0]:
"""
Load the cleaned dataset and compute sentiment scores using NLTK’s VADER analyzer. 
For each review, calculate positive, neutral, negative, and compound sentiment values. 
Display a sample of the results and save the sentiment-enhanced dataset to the Gold layer.
"""

clean_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/clean_train/"
)

nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_pos(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['pos']

def get_sentiment_neu(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['neu']

def get_sentiment_neg(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['neg']

def get_sentiment_compound(text):
    if not text:
        return 0.0
    return analyzer.polarity_scores(text)['compound']

from pyspark.sql import functions as F

sentiment_train = (
    clean_train
    .withColumn("sentiment_pos", F.udf(get_sentiment_pos, DoubleType())("clean_review"))
    .withColumn("sentiment_neu", F.udf(get_sentiment_neu, DoubleType())("clean_review"))
    .withColumn("sentiment_neg", F.udf(get_sentiment_neg, DoubleType())("clean_review"))
    .withColumn("sentiment_compound", F.udf(get_sentiment_compound, DoubleType())("clean_review"))
)

display(
    sentiment_train.select(
        "clean_review",
        "sentiment_pos",
        "sentiment_neu",
        "sentiment_neg",
        "sentiment_compound"
    ).limit(10)
)

sentiment_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/sentiment_train/"
)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


clean_review,sentiment_pos,sentiment_neu,sentiment_neg,sentiment_compound
a fascinating read chalk full of details that benefit from a familiarity with church hierarchy and structure the subtitle is a tad misleading as this books primary focus is the monetary impact of the clergy sex abuse scandal on the catholic church in america with specific attention to the archdioceses of boston cleveland and los angeles but the narrative does extend beyond the us i would recommend it to anyone whos interested in the issues facing the modern church and the need for greater transparency in the institutional structure,0.166,0.77,0.065,0.8735
interesting subject basically it boils down to the vatican expecting the faithful to just pray pay and obey book seemed to ramble in many places would rec,0.245,0.712,0.043,0.7579
the author addresses a history of the finances of the vatican and the catholic with a side trip into sex scandals there is some antidotal information very little positive information is given about church hierarchy and finance leaving me the impression that the author is out to tar the leadership and priests of the church with the human flaws of a few it appears to be well documented and gives some history of the institution also has misinformation about some church fraternal organizations ie the knights of columbus as to their purpose,0.078,0.865,0.057,0.2698
jason berry has delved into the corruption of the rc church from the pope down to the local parish priest he blames the corruption on the unnatural life of celibacy which essentially denies priests an outlet for their sexual inclinations and tends to encourage a gay subculture and opportunities for molestation of the young this situation resulted in the prosecution of many malefactors among the clergy since num and the loss of millions in civil cases special attention is given to pope john pauls favorite mexican born priest maciel who was able to produce huge donations for the pope and himself but who left behind a horrible heap of corruption whatever you thought when you picked the book up you end up knowing its worse than you ever thought possible it helps to have a background in the catholic religion to know the specifics but anyone can follow the story its clearly written although very complex,0.094,0.8,0.106,-0.4497
solid esquerra fun whole time through yes im ignoring those god awful annual bits that were here but i have to say the stories are a bit repetitive strontium dogs come are not liked they look for the criminal they find they fight they collect the reward but the esquerra art is brilliant,0.233,0.531,0.236,-0.0639
the first num ad reprint book ive bought i tend to assume ill have read most of them already by way of titan collections and the various best of num ad reprint mags i loved num ad extreme edition while it lasted even though ive never bought the original mag regularly but this one was all new to me and it was very good what else would you expect from john wagner alan grant and carlos ezquerra brendan mccarthy ian gibson and keith page also contribute the first hundred pages or so are a bit murky being black and white reprints of what i assume were colour pages from starlord but from then on after the strip moved to num ad its lovely and clear and the stories stretch out into real epics an intergalactic war a trip to hell a time travel mission to capture hitler i adored wagner and grants stories for doctor who weekly i wish id known back then that there was practically an entire comic of their work over at num ad every week but its nice getting to read it now,0.161,0.769,0.071,0.9604
olaganustu bir dille yazilmis ki dili icin okumayi yegledigim bir kitapti kullanilan sozcukler ozellikle anlasilirlik acisindan cok guclu ve acik dusun evrenimizi diri tutan bu terimlerin yalnizca arapcalarinifransizcalarini bilenler okurken biraz zorlanabilirler tdk sozlukler url ek kitabin ada yayinlarindan olan ilk baskisinda gecen erimcilik can yayinlari baskisinda

In [0]:
"""
Load the sentiment-enhanced dataset and extract TF-IDF features using a Spark ML pipeline. 
The process includes tokenization, stop word removal, term frequency calculation, 
and inverse document frequency weighting. Cache the result, preview the output, 
and save the TF-IDF feature dataset to the Gold layer.
"""

sentiment_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/sentiment_train/"
)

tokenizer = Tokenizer(inputCol="clean_review", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

cv = CountVectorizer(
    inputCol="filtered_words",
    outputCol="raw_features",
    vocabSize=5000
)

idf = IDF(
    inputCol="raw_features",
    outputCol="tfidf_features"
)

pipeline = Pipeline(stages=[tokenizer, remover, cv, idf])
tfidf_model = pipeline.fit(sentiment_train)
tfidf_train = tfidf_model.transform(sentiment_train)

tfidf_train.cache()

display(tfidf_train.select("clean_review", "tfidf_features").limit(10))

tfidf_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/tfidf_train/"
)


Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

clean_review,tfidf_features
podem ler a opiniao completa aqui even though the anime was never one of my favourites i always wanted to read the manga and so when this got published in portugal i went ahead and bought it im glad i did the drawings are very good the story is as funny as i remember and gokus innocence just gets me everytime ill definetly keep buying,"Map(vectorType -> sparse, length -> 5000, indices -> List(2, 3, 4, 10, 18, 22, 43, 44, 79, 82, 123, 136, 207, 251, 262, 288, 340, 393, 814, 1140, 1467, 2557, 2601, 3291, 4303, 4676), values -> List(0.9612025547308877, 1.1823929571126939, 1.210056347839956, 1.6498846915265137, 1.9310196227317435, 1.9883640321365113, 2.3151545887331344, 2.3554386421428166, 2.605103914115116, 2.648477983445777, 2.9369841880555727, 2.932620123515181, 3.242641736195234, 3.3718816601719106, 3.3939047197491345, 3.4691455531775257, 3.5784689893864083, 3.794172457981501, 4.438930570162933, 4.764611160288203, 5.02632468887679, 6.102889251253811, 5.680619892216889, 6.011222062727987, 6.352895779448671, 6.387793822528478))"
lembrome de ver a serie televisiva quando era mais pequena na sic e quando vi a manga a venda mal quis acreditar d acho que dei pulos de contente devo ter gritado um valente yay e a slayra deve ter pensado ou melhor deve ter tido a certeza que eu havia ensandecido infelizmente so via os volumes mais recentes mas veio a feira e o meu irmao talvez ainda mais doido que eu pela serie la fez o favor de comprar apesar de fa confesso que pouco me lembro de cada episodio tenho mais presente a serie dragon ball z mas pareceme que a manga segue mais ou menos a mesma historia tem as mesmas piadas as mesmas personagens o mesmo desenho nao sera a num a maravilha do mundo mas entretem como os desenhos animados o fizeram em tempos p eu nao me lembrava era que havia tanta piada com cuecas revira os olhos vale o dinheiro gasto talvez nao seja para toda a gente mas va la quem nao cresceu maravilhado com as aventuras do son goku p,"Map(vectorType -> sparse, length -> 5000, indices -> List(1, 31, 47, 54, 59, 268, 362, 407, 503, 526, 587, 768, 788, 887, 907, 1017, 1192, 1226, 1241, 1245, 1278, 1481, 1592, 1736, 1771, 1776, 1973, 2164, 2331, 2557, 2668, 2752, 2773, 2863, 2869, 2882, 2943, 3177, 3411, 3471, 3526, 4241, 4611, 4616), values -> List(1.2438763662225063, 17.83520401268654, 12.318717790652073, 23.751247875375867, 7.340711040256439, 21.374248496459796, 3.9020630433293406, 4.907872562499469, 18.354547180878107, 4.626626420122665, 4.7712214075660855, 4.8232226361070625, 5.2081802767767265, 4.67475148195038, 9.347936147344315, 5.612319002420036, 4.801754610065063, 27.8682674680621, 10.745099763521694, 23.734510057059982, 5.669237336397427, 11.964504931699873, 5.640766053570867, 5.791963294104247, 17.98265427838276, 5.70829303194474, 5.707858318631319, 17.24156046919804, 11.964962458009632, 12.205778502507622, 6.008400523118045, 12.527833135274088, 5.848030335513922, 6.062883073292376, 5.794835453510789, 6.1828638608876485, 5.931740995057982, 6.21304019523427, 6.156789222472924, 19.710895198845954, 6.1853829889225915, 6.2788824870800966, 6.5013467123109, 6.800174012063585))"
spoiler alert individuo sempre gli stessi difetti in questautrice la schone e senza dubbio una scrittrice capace con un suo stile peculiare ma per la terza volta sono costretta a rimarcare le stesse mancanze o forse sarebbe meglio definirle sovrabbondanze forse in questa seconda parte ce meno sesso piu amore e piu dramma rispetto alla prima parte ma si ricade sempre negli stessi difetti sono troppe le scene di sesso gratuito e senza coerenza nel senso del racconto messe li quasi a forza dove invece semplici gesti di tenerezza sarebbero bastati per rendere piu vero il momento profondo e delicato che viene narrato tali scene rendono freddi e poco spontanei alcuni passaggi che emotivamente potevano essere molto intensi e profondi e mi riferisco soprattutto a due scene che francamente ho faticato a tollerare e a mandar giu qua

In [0]:
"""
Load the TF-IDF dataset and generate semantic embeddings using the SentenceTransformer model. 
Each cleaned review is converted into a BERT embedding represented as a list of floats. 
The embeddings are added as a new column and the resulting dataset is saved to the Gold layer.
"""

tfidf_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/tfidf_train/"
)

model_name = "all-MiniLM-L6-v2"

def embed_text(text):
    """
    Given a cleaned review string, return its BERT embedding as a list of floats.
    This function will be wrapped as a Spark UDF to apply on each row in parallel.
    """
    if not text:
        return None
    global model
    try:
        model
    except NameError:
        from sentence_transformers import SentenceTransformer
        model = SentenceTransformer(model_name)
    return model.encode(text).tolist()

embed_udf = udf(embed_text, ArrayType(FloatType()))
embedded_train = tfidf_train.withColumn("bert_embedding", embed_udf("clean_review"))

display(embedded_train.select("clean_review", "bert_embedding").limit(10))

embedded_train.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/embedded_train/"
)


clean_review bert_embedding podem ler a opiniao completa aqui even though the anime was never one of my favourites i always wanted to read the manga and so when this got published in portugal i went ahead and bought it im glad i did the drawings are very good the story is as funny as i remember and gokus innocence just gets me everytime ill definetly keep buying List(-0.0103967, 0.05405688, 0.0052149366, -0.047388636, -0.049518004, 0.029292477, 0.006959799, 0.10609364, 0.06001186, 0.030542014, 0.08989488, -0.004303562, -0.0066768657, 0.026638592, 0.032767013, -0.027320836, 0.027366742, -0.040004212, 0.03924215, -0.059120744, -0.012044889, -0.031548806, 0.05662835, -0.024045855, -0.026402995, 0.011923049, 0.054455023, 0.0065758657, 0.024675997, -0.06088366, -0.037514746, 0.035659872, 0.087417334, -0.02606413, -0.056272697, -0.055646606, 0.0549863, -0.08245922, 0.013511429, 0.07605799, -0.009263884, 0.054266784, -0.022446472, -0.009570799, 0.066139586, 0.008778453, 0.016316188, 0.020475332, 0.007619235, -7.6156843E-4, -0.017077079, 0.0074172504, -0.03957454, -0.029533852, -0.023133505, -0.010195622, -0.010676402, 0.009874415, 0.022179812, -0.034326773, -0.034193147, -0.05440073, -0.014987656, -0.013250143, 0.031145098, -0.08798728, -0.012271153, 0.05642839, -0.019967811, 0.067675486, 0.030927636, -0.03921435, 0.088221684, 0.017074922, -0.077738024, 0.059359718, -0.026108129, -0.053429313, -0.04984919, -0.012774398, -0.03105846, 0.008402864, -0.0011110167, 0.018179014, -0.09968203, 0.0031944436, 0.03125567, -0.009677608, 0.0779529, 0.010322512, 0.08280058, 0.044918936, 0.023055222, 0.05548186, 0.014407596, 0.034102157, 0.049719837, -0.023511745, -0.06537295, 0.031744804, 0.090445586, 0.0028322309, 0.059641466, -0.06949692, -0.0040100105, 0.029420715, 0.07090893, -0.023862278, 0.038919408, 0.05968444, -0.035583697, -0.034782816, 0.0101512065, -0.056523405, -0.03425989, 0.02855248, -0.006116552, -0.034171488, 0.0020295256, -0.009460501, 0.081298076, -0.05325055, 0.015507791, 0.04639021, 0.03534137, -0.12237468, 0.034428265, 2.7488254E-33, -0.036419738, -0.037080143, -0.05874233, 0.01802783, -0.013178964, 0.049799036, 0.015912818, -0.01533582, -0.16687402, -0.011514604, -0.11362622, -0.01070356, -0.18995453, 0.056579206, -0.014037702, 0.052942075, -0.03717599, -0.011785479, 0.052917782, 0.043052956, 0.07367702, -0.024161538, 0.010231958, -0.07239493, -0.022974664, 0.08712071, -0.0036424992, -0.13884817, 0.014236811, 0.041402537, -0.003097877, 0.026231311, -0.06605483, -0.050576106, -0.09299657, -0.06301482, -0.036431767, -0.023864236, -0.037001893, -0.017881503, -7.743393E-4, -0.03697325, -0.044422816, 0.07534288, 0.056889575, 0.047200628, -0.01666589, -0.04134261, 0.08101347, 0.0638079, -0.0982407, -0.086440206, -0.0797555, 0.08310074, 0.022372019, -0.054603405, -0.07687022, -0.003004223, 0.0065188846, -0.029759405, 0.03253709, 0.049320932, 4.633874E-4, 0.0010239754, 0.0062185107, 0.06316861, -0.017648114, -0.005801669, 0.020160679, -0.077568285, -0.069518685, -0.037329454, 0.07048439, -0.041749742, -0.04314122, -0.023765178, -0.029536907, 0.04235036, -0.028004767, 0.0048251683, 0.045009922, -0.014378296, -0.05812882, 0.015121685, -0.016863387, 0.10781424, 0.13967493, -0.044179697, 0.0012995917, 0.024784656, 0.08881144, 0.013305334, 0.04300857, -0.073817566, 0.07059595, -1.7487993E-33, 0.025164897, -0.08408709, -0.08379403, 0.013892824, 0.0012791377, -0.0597604, -0.17049849, 0.06561563, -0.02784019, -0.015525182, -0.065967515, -0.054777473, 0.060765654, 0.012306014, 0.046489075, 0.020963078, 0.11648811, -0.041624576, -6.8898726E-4, -0.029492341, 0.016143365, -0.0071346583, -0.0046445196, -0.015341159, -0.03142613, 0.060734645, 0.057703014, 0.029031105, -0.05079589, 0.06851423, -0.030410018, -0.025384702, -0.018256977, -0.0021132515, 0.024742052, -0.047433626, -6.3547277E-4, 0.005230573, -3.785544E-5, -0.028595885, -0.0047564753, 3.3228213E-4, -0.02173512, -0.00818838, 0.010257798, -0.03472723, -0.009246369, 0.046000767, 0.059871867

In [0]:
"""
Compute the readability score for each review using the Flesch Reading Ease metric from the textstat library. 
Higher scores indicate simpler and easier-to-read text. The result is added as a new column and a sample is displayed.
"""

embedded_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/embedded_train/"
)

def compute_readability(text):
    """
    Compute Flesch Reading Ease score for a given review.
    Returns 0.0 if the text is empty or invalid.
    """
    if not text:
        return 0.0
    try:
        return float(textstat.flesch_reading_ease(text))
    except:
        return 0.0

readability_udf = udf(compute_readability, DoubleType())

embedded_train = embedded_train.withColumn(
    "readability_score", readability_udf("clean_review")
)

display(embedded_train.select("clean_review", "readability_score").limit(5))


clean_review,readability_score
podem ler a opiniao completa aqui even though the anime was never one of my favourites i always wanted to read the manga and so when this got published in portugal i went ahead and bought it im glad i did the drawings are very good the story is as funny as i remember and gokus innocence just gets me everytime ill definetly keep buying,22.42000000000003
lembrome de ver a serie televisiva quando era mais pequena na sic e quando vi a manga a venda mal quis acreditar d acho que dei pulos de contente devo ter gritado um valente yay e a slayra deve ter pensado ou melhor deve ter tido a certeza que eu havia ensandecido infelizmente so via os volumes mais recentes mas veio a feira e o meu irmao talvez ainda mais doido que eu pela serie la fez o favor de comprar apesar de fa confesso que pouco me lembro de cada episodio tenho mais presente a serie dragon ball z mas pareceme que a manga segue mais ou menos a mesma historia tem as mesmas piadas as mesmas personagens o mesmo desenho nao sera a num a maravilha do mundo mas entretem como os desenhos animados o fizeram em tempos p eu nao me lembrava era que havia tanta piada com cuecas revira os olhos vale o dinheiro gasto talvez nao seja para toda a gente mas va la quem nao cresceu maravilhado com as aventuras do son goku p,-105.58499999999998
spoiler alert individuo sempre gli stessi difetti in questautrice la schone e senza dubbio una scrittrice capace con un suo stile peculiare ma per la terza volta sono costretta a rimarcare le stesse mancanze o forse sarebbe meglio definirle sovrabbondanze forse in questa seconda parte ce meno sesso piu amore e piu dramma rispetto alla prima parte ma si ricade sempre negli stessi difetti sono troppe le scene di sesso gratuito e senza coerenza nel senso del racconto messe li quasi a forza dove invece semplici gesti di tenerezza sarebbero bastati per rendere piu vero il momento profondo e delicato che viene narrato tali scene rendono freddi e poco spontanei alcuni passaggi che emotivamente potevano essere molto intensi e profondi e mi riferisco soprattutto a due scene che francamente ho faticato a tollerare e a mandar giu quando fanno sesso mentre parlano dellimputata difesa da james che si e impiccata e tutta la scena che si svolge nellaula di tribunale vuota scene davvero al limite del buongusto ed e a causa di cio delle esagerazioni nel voler inserire il sesso anche li dove potrebbe farne a meno che si abbassa il mio giudizio su un romanzo che invece nella sua interezza risulta anche interessante e ben scritto e che affronta argomenti delicati e importanti come lemancipazione delle donne nel num deg secolo e le loro difficolta e rivendicazioni in una societa concepita solo dal punto di vista maschile ribadisco poi anche linsensatezza della scelta della mondadori di pubblicare prima il grido del desiderio che ha fatto perdere a chi come me lo ha letto prima di questo romanzo il senso e le emozioni della narrazione delle vicende di frances e james,-220.18284697508892
espiral es la primera novela grafica de bef quien hasta poco antes habia publicado mucha narrativa breve grafica sus dos compilaciones monorama tienen mucho de lo mejor que hizo despues de espiral ademas de seguir con sus breverias ilustro el guion original de juan villoro la calavera de cristal para sexto piso y la version de fg haghenbeck de los bandidos de rio frio cuando salio este libro escribi una resena en mi blog de lecturas ritual habitual que aqui les dejo para que lean los interesados resena de espiral en ritual habitual,-34.018064516129
una obra dotada de originalidad por donde se vea que se disfruta leyendo en cada uno de sus juegos metaficcionales que impresionan a veces me hacian soltar una risa complice del objeto dentro del objeto en un momento ya no me parecio tan atractivo de donde iba saliendo cada historia esa impresion ya no era del todo llamativa sin embargo si se volteaba a ver atras de donde habia salido todas las historias resultaba u

In [0]:
"""
Compute the subjectivity score for each review using TextBlob sentiment analysis. 
Values range from 0 (objective and factual) to 1 (highly subjective or opinion-based). 
The score is added as a new column and a sample of results is displayed.
"""

def compute_subjectivity(text):
    """
    Compute subjectivity score using TextBlob.
    Returns 0.0 for empty or invalid text.
    """
    if not text:
        return 0.0
    return float(TextBlob(text).sentiment.subjectivity)

subjectivity_udf = udf(compute_subjectivity, DoubleType())

embedded_train = embedded_train.withColumn(
    "subjectivity", subjectivity_udf("clean_review")
)

display(embedded_train.select("clean_review", "subjectivity").limit(5))


clean_review,subjectivity
podem ler a opiniao completa aqui even though the anime was never one of my favourites i always wanted to read the manga and so when this got published in portugal i went ahead and bought it im glad i did the drawings are very good the story is as funny as i remember and gokus innocence just gets me everytime ill definetly keep buying,0.9450000000000001
lembrome de ver a serie televisiva quando era mais pequena na sic e quando vi a manga a venda mal quis acreditar d acho que dei pulos de contente devo ter gritado um valente yay e a slayra deve ter pensado ou melhor deve ter tido a certeza que eu havia ensandecido infelizmente so via os volumes mais recentes mas veio a feira e o meu irmao talvez ainda mais doido que eu pela serie la fez o favor de comprar apesar de fa confesso que pouco me lembro de cada episodio tenho mais presente a serie dragon ball z mas pareceme que a manga segue mais ou menos a mesma historia tem as mesmas piadas as mesmas personagens o mesmo desenho nao sera a num a maravilha do mundo mas entretem como os desenhos animados o fizeram em tempos p eu nao me lembrava era que havia tanta piada com cuecas revira os olhos vale o dinheiro gasto talvez nao seja para toda a gente mas va la quem nao cresceu maravilhado com as aventuras do son goku p,0.0
spoiler alert individuo sempre gli stessi difetti in questautrice la schone e senza dubbio una scrittrice capace con un suo stile peculiare ma per la terza volta sono costretta a rimarcare le stesse mancanze o forse sarebbe meglio definirle sovrabbondanze forse in questa seconda parte ce meno sesso piu amore e piu dramma rispetto alla prima parte ma si ricade sempre negli stessi difetti sono troppe le scene di sesso gratuito e senza coerenza nel senso del racconto messe li quasi a forza dove invece semplici gesti di tenerezza sarebbero bastati per rendere piu vero il momento profondo e delicato che viene narrato tali scene rendono freddi e poco spontanei alcuni passaggi che emotivamente potevano essere molto intensi e profondi e mi riferisco soprattutto a due scene che francamente ho faticato a tollerare e a mandar giu quando fanno sesso mentre parlano dellimputata difesa da james che si e impiccata e tutta la scena che si svolge nellaula di tribunale vuota scene davvero al limite del buongusto ed e a causa di cio delle esagerazioni nel voler inserire il sesso anche li dove potrebbe farne a meno che si abbassa il mio giudizio su un romanzo che invece nella sua interezza risulta anche interessante e ben scritto e che affronta argomenti delicati e importanti come lemancipazione delle donne nel num deg secolo e le loro difficolta e rivendicazioni in una societa concepita solo dal punto di vista maschile ribadisco poi anche linsensatezza della scelta della mondadori di pubblicare prima il grido del desiderio che ha fatto perdere a chi come me lo ha letto prima di questo romanzo il senso e le emozioni della narrazione delle vicende di frances e james,0.375
espiral es la primera novela grafica de bef quien hasta poco antes habia publicado mucha narrativa breve grafica sus dos compilaciones monorama tienen mucho de lo mejor que hizo despues de espiral ademas de seguir con sus breverias ilustro el guion original de juan villoro la calavera de cristal para sexto piso y la version de fg haghenbeck de los bandidos de rio frio cuando salio este libro escribi una resena en mi blog de lecturas ritual habitual que aqui les dejo para que lean los interesados resena de espiral en ritual habitual,0.75
una obra dotada de originalidad por donde se vea que se disfruta leyendo en cada uno de sus juegos metaficcionales que impresionan a veces me hacian soltar una risa complice del objeto dentro del objeto en un momento ya no me parecio tan atractivo de donde iba saliendo cada historia esa impresion ya no era del todo llamativa sin embargo si se volteaba a ver atras de donde habia salido todas las historias resultaba una cosa increible entre sorpresa y gusto por l

In [0]:
"""
Calculate the average word length for each review to estimate language complexity. 
Longer words suggest more complex writing, while shorter words indicate simpler text. 
The computed values are added as a new column and a sample is displayed.
"""

def compute_avg_word_length(text):
    """
    Calculate average word length for a given review.
    Returns 0.0 for empty text.
    """
    if not text:
        return 0.0
    words = text.split()
    return float(np.mean([len(w) for w in words])) if words else 0.0

avg_word_len_udf = udf(compute_avg_word_length, DoubleType())

embedded_train = embedded_train.withColumn(
    "avg_word_length", avg_word_len_udf("clean_review")
)

display(embedded_train.select("clean_review", "avg_word_length").limit(5))


clean_review,avg_word_length
podem ler a opiniao completa aqui even though the anime was never one of my favourites i always wanted to read the manga and so when this got published in portugal i went ahead and bought it im glad i did the drawings are very good the story is as funny as i remember and gokus innocence just gets me everytime ill definetly keep buying,4.184615384615385
lembrome de ver a serie televisiva quando era mais pequena na sic e quando vi a manga a venda mal quis acreditar d acho que dei pulos de contente devo ter gritado um valente yay e a slayra deve ter pensado ou melhor deve ter tido a certeza que eu havia ensandecido infelizmente so via os volumes mais recentes mas veio a feira e o meu irmao talvez ainda mais doido que eu pela serie la fez o favor de comprar apesar de fa confesso que pouco me lembro de cada episodio tenho mais presente a serie dragon ball z mas pareceme que a manga segue mais ou menos a mesma historia tem as mesmas piadas as mesmas personagens o mesmo desenho nao sera a num a maravilha do mundo mas entretem como os desenhos animados o fizeram em tempos p eu nao me lembrava era que havia tanta piada com cuecas revira os olhos vale o dinheiro gasto talvez nao seja para toda a gente mas va la quem nao cresceu maravilhado com as aventuras do son goku p,4.144444444444445
spoiler alert individuo sempre gli stessi difetti in questautrice la schone e senza dubbio una scrittrice capace con un suo stile peculiare ma per la terza volta sono costretta a rimarcare le stesse mancanze o forse sarebbe meglio definirle sovrabbondanze forse in questa seconda parte ce meno sesso piu amore e piu dramma rispetto alla prima parte ma si ricade sempre negli stessi difetti sono troppe le scene di sesso gratuito e senza coerenza nel senso del racconto messe li quasi a forza dove invece semplici gesti di tenerezza sarebbero bastati per rendere piu vero il momento profondo e delicato che viene narrato tali scene rendono freddi e poco spontanei alcuni passaggi che emotivamente potevano essere molto intensi e profondi e mi riferisco soprattutto a due scene che francamente ho faticato a tollerare e a mandar giu quando fanno sesso mentre parlano dellimputata difesa da james che si e impiccata e tutta la scena che si svolge nellaula di tribunale vuota scene davvero al limite del buongusto ed e a causa di cio delle esagerazioni nel voler inserire il sesso anche li dove potrebbe farne a meno che si abbassa il mio giudizio su un romanzo che invece nella sua interezza risulta anche interessante e ben scritto e che affronta argomenti delicati e importanti come lemancipazione delle donne nel num deg secolo e le loro difficolta e rivendicazioni in una societa concepita solo dal punto di vista maschile ribadisco poi anche linsensatezza della scelta della mondadori di pubblicare prima il grido del desiderio che ha fatto perdere a chi come me lo ha letto prima di questo romanzo il senso e le emozioni della narrazione delle vicende di frances e james,4.95729537366548
espiral es la primera novela grafica de bef quien hasta poco antes habia publicado mucha narrativa breve grafica sus dos compilaciones monorama tienen mucho de lo mejor que hizo despues de espiral ademas de seguir con sus breverias ilustro el guion original de juan villoro la calavera de cristal para sexto piso y la version de fg haghenbeck de los bandidos de rio frio cuando salio este libro escribi una resena en mi blog de lecturas ritual habitual que aqui les dejo para que lean los interesados resena de espiral en ritual habitual,4.78494623655914
una obra dotada de originalidad por donde se vea que se disfruta leyendo en cada uno de sus juegos metaficcionales que impresionan a veces me hacian soltar una risa complice del objeto dentro del objeto en un momento ya no me parecio tan atractivo de donde iba saliendo cada historia esa impresion ya no era del todo llamativa sin embargo si se volteaba a ver atras de donde habia salido todas las historias resultaba una cosa

In [0]:
"""
Compute the lexical diversity of each review to measure vocabulary richness. 
It is calculated as the number of unique words divided by the total number of words. 
Higher values indicate more varied language use. The results are added as a new column and a sample is displayed.
"""

def compute_lexical_diversity(text):
    """
    Compute lexical diversity for a given review.
    Returns 0.0 for empty text.
    """
    if not text:
        return 0.0
    words = text.split()
    unique_words = set(words)
    return float(len(unique_words) / len(words)) if words else 0.0

lexical_div_udf = udf(compute_lexical_diversity, DoubleType())

embedded_train = embedded_train.withColumn(
    "lexical_diversity", lexical_div_udf("clean_review")
)

display(embedded_train.select("clean_review", "lexical_diversity").limit(5))


clean_review,lexical_diversity
podem ler a opiniao completa aqui even though the anime was never one of my favourites i always wanted to read the manga and so when this got published in portugal i went ahead and bought it im glad i did the drawings are very good the story is as funny as i remember and gokus innocence just gets me everytime ill definetly keep buying,0.8615384615384616
lembrome de ver a serie televisiva quando era mais pequena na sic e quando vi a manga a venda mal quis acreditar d acho que dei pulos de contente devo ter gritado um valente yay e a slayra deve ter pensado ou melhor deve ter tido a certeza que eu havia ensandecido infelizmente so via os volumes mais recentes mas veio a feira e o meu irmao talvez ainda mais doido que eu pela serie la fez o favor de comprar apesar de fa confesso que pouco me lembro de cada episodio tenho mais presente a serie dragon ball z mas pareceme que a manga segue mais ou menos a mesma historia tem as mesmas piadas as mesmas personagens o mesmo desenho nao sera a num a maravilha do mundo mas entretem como os desenhos animados o fizeram em tempos p eu nao me lembrava era que havia tanta piada com cuecas revira os olhos vale o dinheiro gasto talvez nao seja para toda a gente mas va la quem nao cresceu maravilhado com as aventuras do son goku p,0.6722222222222223
spoiler alert individuo sempre gli stessi difetti in questautrice la schone e senza dubbio una scrittrice capace con un suo stile peculiare ma per la terza volta sono costretta a rimarcare le stesse mancanze o forse sarebbe meglio definirle sovrabbondanze forse in questa seconda parte ce meno sesso piu amore e piu dramma rispetto alla prima parte ma si ricade sempre negli stessi difetti sono troppe le scene di sesso gratuito e senza coerenza nel senso del racconto messe li quasi a forza dove invece semplici gesti di tenerezza sarebbero bastati per rendere piu vero il momento profondo e delicato che viene narrato tali scene rendono freddi e poco spontanei alcuni passaggi che emotivamente potevano essere molto intensi e profondi e mi riferisco soprattutto a due scene che francamente ho faticato a tollerare e a mandar giu quando fanno sesso mentre parlano dellimputata difesa da james che si e impiccata e tutta la scena che si svolge nellaula di tribunale vuota scene davvero al limite del buongusto ed e a causa di cio delle esagerazioni nel voler inserire il sesso anche li dove potrebbe farne a meno che si abbassa il mio giudizio su un romanzo che invece nella sua interezza risulta anche interessante e ben scritto e che affronta argomenti delicati e importanti come lemancipazione delle donne nel num deg secolo e le loro difficolta e rivendicazioni in una societa concepita solo dal punto di vista maschile ribadisco poi anche linsensatezza della scelta della mondadori di pubblicare prima il grido del desiderio che ha fatto perdere a chi come me lo ha letto prima di questo romanzo il senso e le emozioni della narrazione delle vicende di frances e james,0.6690391459074733
espiral es la primera novela grafica de bef quien hasta poco antes habia publicado mucha narrativa breve grafica sus dos compilaciones monorama tienen mucho de lo mejor que hizo despues de espiral ademas de seguir con sus breverias ilustro el guion original de juan villoro la calavera de cristal para sexto piso y la version de fg haghenbeck de los bandidos de rio frio cuando salio este libro escribi una resena en mi blog de lecturas ritual habitual que aqui les dejo para que lean los interesados resena de espiral en ritual habitual,0.7419354838709677
una obra dotada de originalidad por donde se vea que se disfruta leyendo en cada uno de sus juegos metaficcionales que impresionan a veces me hacian soltar una risa complice del objeto dentro del objeto en un momento ya no me parecio tan atractivo de donde iba saliendo cada historia esa impresion ya no era del todo llamativa sin embargo si se volteaba a ver atras de donde habia salido todas las historias resultaba 

In [0]:
"""
Load the embedded dataset from the Gold layer and save it as a combined dataset. 
This step consolidates all previously engineered features into a single Delta table 
for easier access in later analysis or model training.
"""

embedded_train = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/embedded_train/"
)

combined_train = embedded_train
combined_train.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("abfss://lakehouse@goodreadsreviews60106541.dfs.core.windows.net/gold/feature_v2/combined_train/")


In [0]:
display(combined_train.limit(10))

book_id review_id title author_id name user_id rating review_text language n_votes date_added review_length_words avg_rating num_reviews clean_review review_length_chars sentiment_pos sentiment_neu sentiment_neg sentiment_compound words filtered_words raw_features tfidf_features bert_embedding 10001021 16b1077b60e3674e4edaef7951194c43 Dragon Ball, Vol. 1: Son Goku (Dragon Ball, #1) 8841094 hellodickspeight ebc700f4f7abb6203ef6f8d987d80e40 4.0 podem ler a opiniao completa aqui. 
 even though the anime was never one of my favourites, i always wanted to read the manga, and so when this got published in portugal, i went ahead and bought it. im glad i did! 
 the drawings are very good, the story is as funny as i remember, and gokus innocence just gets me everytime. 
 ill definetly keep buying. por 28 2010-10-01 65 3.75 4 podem ler a opiniao completa aqui even though the anime was never one of my favourites i always wanted to read the manga and so when this got published in portugal i went ahead and bought it im glad i did the drawings are very good the story is as funny as i remember and gokus innocence just gets me everytime ill definetly keep buying 336 0.168 0.791 0.04 0.8357 List(podem, ler, a, opiniao, completa, aqui, even, though, the, anime, was, never, one, of, my, favourites, i, always, wanted, to, read, the, manga, and, so, when, this, got, published, in, portugal, i, went, ahead, and, bought, it, im, glad, i, did, the, drawings, are, very, good, the, story, is, as, funny, as, i, remember, and, gokus, innocence, just, gets, me, everytime, ill, definetly, keep, buying) List(podem, ler, opiniao, completa, aqui, even, though, anime, never, one, favourites, always, wanted, read, manga, got, published, portugal, went, ahead, bought, im, glad, drawings, good, story, funny, remember, gokus, innocence, gets, everytime, ill, definetly, keep, buying) Map(vectorType -> sparse, length -> 5000, indices -> List(2, 3, 4, 10, 18, 22, 43, 44, 79, 82, 123, 136, 207, 251, 262, 288, 340, 393, 814, 1140, 1467, 2557, 2601, 3291, 4303, 4676), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)) Map(vectorType -> sparse, length -> 5000, indices -> List(2, 3, 4, 10, 18, 22, 43, 44, 79, 82, 123, 136, 207, 251, 262, 288, 340, 393, 814, 1140, 1467, 2557, 2601, 3291, 4303, 4676), values -> List(0.9612025547308877, 1.1823929571126939, 1.210056347839956, 1.6498846915265137, 1.9310196227317435, 1.9883640321365113, 2.3151545887331344, 2.3554386421428166, 2.605103914115116, 2.648477983445777, 2.9369841880555727, 2.932620123515181, 3.242641736195234, 3.3718816601719106, 3.3939047197491345, 3.4691455531775257, 3.5784689893864083, 3.794172457981501, 4.438930570162933, 4.764611160288203, 5.02632468887679, 6.102889251253811, 5.680619892216889, 6.011222062727987, 6.352895779448671, 6.387793822528478)) List(-0.0103967, 0.05405688, 0.0052149366, -0.047388636, -0.049518004, 0.029292477, 0.006959799, 0.10609364, 0.06001186, 0.030542014, 0.08989488, -0.004303562, -0.0066768657, 0.026638592, 0.032767013, -0.027320836, 0.027366742, -0.040004212, 0.03924215, -0.059120744, -0.012044889, -0.031548806, 0.05662835, -0.024045855, -0.026402995, 0.011923049, 0.054455023, 0.0065758657, 0.024675997, -0.06088366, -0.037514746, 0.035659872, 0.087417334, -0.02606413, -0.056272697, -0.055646606, 0.0549863, -0.08245922, 0.013511429, 0.07605799, -0.009263884, 0.054266784, -0.022446472, -0.009570799, 0.066139586, 0.008778453, 0.016316188, 0.020475332, 0.007619235, -7.6156843E-4, -0.017077079, 0.0074172504, -0.03957454, -0.029533852, -0.023133505, -0.010195622, -0.010676402, 0.009874415, 0.022179812, -0.034326773, -0.034193147, -0.05440073, -0.014987656, -0.013250143, 0.031145098, -0.08798728, -0.012271153, 0.05642839, -0.019967811, 0.067675486, 0.030927636, -0.03921435, 0.088221684, 0.017074922, -0.077738024, 0.059359718, -0.026108129, -0.053429313, -0.04984919, -0.012774398, -0.03105846, 0.00